In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt 
%matplotlib inline
import tensorflow as tf
from tensorflow import keras 
import numpy as np
import sklearn
import pandas as pd
import sys, os, shutil
print(tf.__version__)
print(sys.version_info)

2.0.0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


## Load Data

In [2]:
fashion_mnist = keras.datasets.fashion_mnist #https://www.tensorflow.org/datasets/catalog/fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

## Dimensionality checking

In [3]:
print("training set size:")
print(x_train.shape, y_train.shape)
print("validation set size:")
print(x_valid.shape, y_valid.shape)
print("testing set size:")
print(x_test.shape, y_test.shape)

training set size:
(55000, 28, 28) (55000,)
validation set size:
(5000, 28, 28) (5000,)
testing set size:
(10000, 28, 28) (10000,)


## Standardization

In [4]:
# https://scikit-learn.org/stable/modules/preprocessing.html
# x = (x-u)/std to make x with mean 0 and std 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
# use the mean and std of training set to do standardization for validation/testing set
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

## Build the model with keras.Sequential API (w/ batch normalization and dropout)

In [5]:
# relu: y=max(0, x)
# softmax: x = [2, 3, 8]
#          sum = e^2 + e^3 + e^8
#          y = [e^2/sum, e^3/sum, e^8/sum] sum up to 1 (as probability)

#https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/AlphaDropout
#https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28])) # 28x28 image -> 784
model.add(keras.layers.Dense(300, activation="relu")) #fully connected layer 1
model.add(keras.layers.BatchNormalization()) #way 1: BN after activation
"""
model.add(keras.layers.Dense(300)) #fully connected layer 1 (without activation)
model.add(keras.layers.BatchNormalization()) #way 2: BN before activation
model.add(keras.layers.Activation('relu')) # activation after batch norm layer
"""
model.add(keras.layers.AlphaDropout(rate=0.3)) #dropout layer
model.add(keras.layers.Dense(100, activation="relu")) #fully connected layer 2
model.add(keras.layers.Dense(10, activation="softmax")) # output layer

## Compile the model


In [ ]:
#reference links
# cross entropy
#https://www.tensorflow.org/api_docs/python/tf/keras/losses/sparse_categorical_crossentropy?hl=en
# sgd
#https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD
model.compile(loss="sparse_categorical_crossentropy",
				optimizer=keras.optimizers.SGD(0.001),
				metrics=["accuracy"])
model.summary()

## callback functions

In [ ]:
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks
# keras.callbacks.Tensorboard: https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/TensorBoard
# keras.callbacks.ModelCheckpoint: https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/ModelCheckpoint
# keras.callbacks.EarlyStopping: https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/EarlyStopping
# check tensorboard record: tensorboard --logdir=Log_Dir_You_Saved_The_Logs

logdir = "/Users/yantinghuang/Study/BIOS534_Yu_spring2019/log_dir_classification" # change to your preferred directory path
if not os.path.exists(logdir):
	os.mkdir(logdir)
output_model_fn = os.path.join(logdir, "fashion_mnist_model.h5")
callbacks = [
		keras.callbacks.TensorBoard(logdir),
		keras.callbacks.ModelCheckpoint(output_model_fn,save_best_only=True),
		keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)]

## Model Training

In [ ]:
history = model.fit(x_train_scaled, y_train, epochs=10,
	validation_data=(x_valid_scaled, y_valid),
	callbacks=callbacks)
print(history.history)


## Plot training history

In [ ]:
def plot_learning_curves(history):
	pd.DataFrame(history.history).plot(figsize=(8, 5))
	plt.grid(True)
	plt.gca().set_ylim(0, 1)
	plt.show()
plot_learning_curves(history)